In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets
from torch.utils import data

In [2]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [3]:
EPOCHS = 30
BATCH_SIZE = 64

In [4]:
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,))
])
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,))
])

In [5]:
trainset = datasets.FashionMNIST(
    root = './.data/',
    train = True,
    transform = train_transform,
    download=True
)

In [6]:
testset = datasets.FashionMNIST(
    root = './.data/',
    train = False,
    transform = test_transform,
    download=True
)

In [7]:
train_loader = data.DataLoader(
    dataset = trainset,
    batch_size=BATCH_SIZE)
test_loader = data.DataLoader(
    dataset = testset,
    batch_size= BATCH_SIZE
)

In [8]:
class Net(nn.Module):
    def __init__(self, dropout_p=0.2):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128,10)
        self.dropout_p = dropout_p
        
    def forward(self, input):
        flatten = input.view(-1, 784)
        dropped1 = F.dropout(flatten, training=self.training, p=self.dropout_p )
        fc1 = F.relu(self.fc1(dropped1))
        dropped2 = F.dropout(fc1, training=self.training, p=self.dropout_p)
        fc2 = F.relu(self.fc2(dropped2))
        output = self.fc3(fc2)
        return output

In [9]:
model = Net(dropout_p=0.2).to(DEVICE)

In [10]:
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [11]:
def train(model, train_loader, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
    
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

In [12]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
        
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100 * correct / len(test_loader.dataset)
    
    return test_loss, test_accuracy

In [ ]:
test   = torch.Tensor([[0,0,1,0,0,0,0,0,0],[0,0,0,0,1,0,0,0,0],[1,0,0,0,0,0,0,0,0]])

In [ ]:
test.max(1)

In [ ]:
test.max(1, keepdim=True)[1].shape

In [13]:
for epoch in range(1, EPOCHS+1):
    train(model, train_loader, optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print(f'{epoch:2d} Test Loss : {test_loss:.4f}, accuracy : {test_accuracy:.2f}')

 1 Test Loss : 0.6482, accuracy : 76.79
 2 Test Loss : 0.5410, accuracy : 80.34
 3 Test Loss : 0.5004, accuracy : 81.78
 4 Test Loss : 0.4780, accuracy : 82.32
 5 Test Loss : 0.4529, accuracy : 83.45
 6 Test Loss : 0.4414, accuracy : 83.80
 7 Test Loss : 0.4265, accuracy : 84.30
 8 Test Loss : 0.4148, accuracy : 85.00
 9 Test Loss : 0.4108, accuracy : 85.04
10 Test Loss : 0.4046, accuracy : 85.23
11 Test Loss : 0.3979, accuracy : 85.37
12 Test Loss : 0.3913, accuracy : 85.76
13 Test Loss : 0.3893, accuracy : 85.85
14 Test Loss : 0.3830, accuracy : 86.08
15 Test Loss : 0.3783, accuracy : 86.03
16 Test Loss : 0.3763, accuracy : 86.20
17 Test Loss : 0.3688, accuracy : 86.53
18 Test Loss : 0.3641, accuracy : 86.66
19 Test Loss : 0.3604, accuracy : 87.02
20 Test Loss : 0.3615, accuracy : 86.90
21 Test Loss : 0.3559, accuracy : 86.90
22 Test Loss : 0.3543, accuracy : 87.13
23 Test Loss : 0.3515, accuracy : 87.22
24 Test Loss : 0.3480, accuracy : 87.35
25 Test Loss : 0.3432, accuracy : 87.64
